# opt.dir 分别改成train和test

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
f_img   = '/disk1/home/yupeng/Data/K3OD/training/image_2/000111.png'
f_label   = '/disk1/home/yupeng/Data/K3OD/training/label_2/000111.txt'
# imgooo = mpimg.imread(f_img) 
# plt.imshow(imgooo)

In [ ]:
lines = open(f_label).readlines()
lines
# lines = [x.strip('\n') for x in lines]
items = [x.strip(' ').split(' ') for x in lines]
items[1][]
# items[0][4:8]
# lines

In [ ]:
label_dir = '/disk1/home/yupeng/Data/K3OD/training/label_2/'
labels = []
for frame in range(0, 7481):

    label_f = label_dir + '%06d.txt' % frame
#     print(label_f)
    lines = open(label_f).readlines()
    lines = [x.strip('\n') for x in lines]
    items = [x.strip(' ').split(' ') for x in lines]
#     print(len(lines))
    for j in range(len(lines)):
        new_label = items[j][0]
        if new_label not in labels:
            labels.append(new_label)
            print(new_label)

In [ ]:
labels

# 需要 BBox ('ymin', 'xmin', 'ymax', 'xmax')


### 目前：第5-8列的599.41 156.40 629.75 189.25是目标的2D bounding box 像素位置，形式为xyxy，前两个值为bounding box左上点的x，y位置，后两个点为右下角的x,y位置。

In [ ]:


bbox = list()
label = list()
difficult = list()
depth = list()
y_rot = list()

xmin, ymin, xmax, ymax = items[0][4:8]
bbox.append([int(float(ymin)), int(float(xmin)), int(float(ymax)), int(float(xmax))])
label.append(items[0][0])
difficult.append(False)
depth.append(float(items[0][13]))
y_rot.append(float(items[0][3]))

bbox = np.stack(bbox).astype(np.float32)
label = np.stack(label).astype(np.int32)
print(type(difficult))
difficult = np.array(difficult, dtype=np.bool).astype(np.uint8)
print(type(difficult))

In [ ]:
import torch as t
def pytorch_normalze(img):
    """
    https://github.com/pytorch/vision/issues/223
    return appr -1~1 RGB
    """
    normalize = tvtsf.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
    img = normalize(t.from_numpy(img))
    return img.numpy()

test inverse_normalize

In [ ]:
f = Image.open(f_img)
img = f.convert('RGB')
img = np.asarray(img, dtype=np.float32)
img = img.transpose((2, 0, 1))
img.shape
img = img /255

In [ ]:
print(img.min())
print(img.max())

In [ ]:
pn_img = pytorch_normalze(img)
print(pn_img.max())
print(pn_img.min())

In [ ]:
_, H, W = img.shape
img, params = util.random_flip(img, x_random=True, return_param=True)
_, o_H, o_W = img.shape
scale = o_H / H

In [ ]:
bbox = util.flip_bbox(bbox, (o_H, o_W), x_flip=params['x_flip'])

In [ ]:
# bbox (~numpy.ndarray): An array whose shape is :math:`(R, 4)`.
#             :math:`R` is the number of bounding boxes.

In [ ]:
bbox

In [ ]:

import os
import numpy as np
from data.util import read_image

In [ ]:
a = ImageKITTIDataset('/disk1/home/yupeng/Data/temp2D/test')

In [ ]:
ori_img, bbox, label, difficult, depth, y_rot = a.get_example(5)

In [ ]:
depth

In [2]:
from __future__ import  absolute_import
from __future__ import  division
import torch as t
from skimage import transform as sktsf
from torchvision import transforms as tvtsf
from data import util
import numpy as np
from utils.config import opt
from math import pi
from data.kitti_img_dataset import ImageKITTIDataset

In [ ]:
def inverse_normalize(img):
    # approximate un-normalize for visualize
    return (img * 0.225 + 0.45).clip(min=0, max=1) * 255


def pytorch_normalze(img):
    """
    https://github.com/pytorch/vision/issues/223
    return appr -1~1 RGB
    """
    normalize = tvtsf.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
    img = normalize(t.from_numpy(img))
    return img.numpy()

def preprocess(img, min_size=375, max_size=1242):
    """Preprocess an image for feature extraction.
    The length of the shorter edge is scaled to :obj:`self.min_size`.
    After the scaling, if the length of the longer edge is longer than
    :param min_size:
    :obj:`self.max_size`, the image is scaled to fit the longer edge
    to :obj:`self.max_size`.
    After resizing the image, the image is subtracted by a mean image value
    :obj:`self.mean`.
    Args:
        img (~numpy.ndarray): An image. This is in CHW and RGB format.
            The range of its value is :math:`[0, 255]`.
    Returns:
        ~numpy.ndarray: A preprocessed image.
    """
    C, H, W = img.shape
    scale1 = min_size / min(H, W)
    scale2 = max_size / max(H, W)
    scale = min(scale1, scale2)
    img = img / 255.
    img = sktsf.resize(img, (C, H * scale, W * scale), mode='reflect',anti_aliasing=False)
    # both the longer and shorter should be less than
    # max_size and min_size
    normalize = pytorch_normalze
    return normalize(img)

class Transform(object):

    def __init__(self, min_size=375, max_size=1242):
        self.min_size = min_size
        self.max_size = max_size

    def __call__(self, in_data):
        img, bbox, label, depth, y_rot = in_data
        _, H, W = img.shape
        img = preprocess(img, self.min_size, self.max_size)
        _, o_H, o_W = img.shape
        scale = o_H / H
        bbox = util.resize_bbox(bbox, (H, W), (o_H, o_W))

        # horizontally flip
        img, params = util.random_flip(
            img, x_random=True, return_param=True)
        bbox = util.flip_bbox(
            bbox, (o_H, o_W), x_flip=params['x_flip'])
        print(params['x_flip'])
        if params['x_flip']:
            print(y_rot)
            for i in range(len(y_rot)):
                theta = float(y_rot[i])
                if theta > 0:
                    y_rot[i] = pi - theta
                if theta < 0:
                    y_rot[i] = -pi - theta
                    y_rot[i]
                if theta == 0:
                    y_rot[i] = 3.14

        return img, bbox, label, depth, y_rot, scale

In [3]:
db = ImageKITTIDataset('/disk1/home/yupeng/Data/temp2D/test')

In [4]:
ori_img, bbox, label, difficult, depth, y_rot = db.get_example(10)

000066


In [5]:
ori_img, bbox, label, difficult, depth, y_rot

(array([[[109., 110., 123., ...,   8.,   8.,   6.],
         [110., 108., 118., ...,   4.,   5.,   6.],
         [118., 107., 164., ...,   6.,   7.,   7.],
         ...,
         [ 15.,  15.,  16., ...,  12.,  13.,  13.],
         [ 16.,  16.,  16., ...,  12.,  12.,  13.],
         [ 14.,  14.,  15., ...,  13.,  13.,  13.]],
 
        [[177., 178., 178., ...,   8.,   6.,   6.],
         [175., 176., 180., ...,   6.,   7.,  10.],
         [172., 175., 177., ...,   8.,  13.,  21.],
         ...,
         [ 17.,  17.,  18., ...,  13.,  13.,  11.],
         [ 20.,  20.,  20., ...,  14.,  13.,  12.],
         [ 18.,  18.,  18., ...,  15.,  14.,  13.]],
 
        [[255., 255., 255., ...,   5.,   6.,   6.],
         [255., 255., 255., ...,   6.,   6.,   6.],
         [255., 255., 255., ...,   7.,   6.,   6.],
         ...,
         [ 15.,  17.,  20., ...,  13.,  13.,  13.],
         [ 24.,  26.,  27., ...,  13.,  13.,  13.],
         [ 27.,  27.,  27., ...,  13.,  13.,  13.]]], dtype=float32)

In [10]:
img = preprocess(ori_img)

In [9]:
def preprocess(img, min_size=375, max_size=1242):
    """Preprocess an image for feature extraction.
    The length of the shorter edge is scaled to :obj:`self.min_size`.
    After the scaling, if the length of the longer edge is longer than
    :param min_size:
    :obj:`self.max_size`, the image is scaled to fit the longer edge
    to :obj:`self.max_size`.
    After resizing the image, the image is subtracted by a mean image value
    :obj:`self.mean`.
    Args:
        img (~numpy.ndarray): An image. This is in CHW and RGB format.
            The range of its value is :math:`[0, 255]`.
    Returns:
        ~numpy.ndarray: A preprocessed image.
    """
    C, H, W = img.shape
    scale1 = min_size / min(H, W)
    scale2 = max_size / max(H, W)
    scale = min(scale1, scale2)
    img = img / 255.
    img = sktsf.resize(img, (C, H * scale, W * scale), mode='reflect',anti_aliasing=False)
    # both the longer and shorter should be less than
    # max_size and min_size
    normalize = pytorch_normalze
    return normalize(img)
def pytorch_normalze(img):
    """
    https://github.com/pytorch/vision/issues/223
    return appr -1~1 RGB
    """
    normalize = tvtsf.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
    img = normalize(t.from_numpy(img))
    return img.numpy()

(375, 1242)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
ori_img_t = ori_img/255
plt.imshow(ori_img_t.transpose((1, 2, 0)))

In [ ]:
nimg_t = inverse_normalize(nimg)/255
plt.imshow(nimg_t.transpose((1, 2, 0)))

In [ ]:
print(bbox)
test_img = ori_img[:, 168:206,  671:710]
test_img_t = test_img/255
plt.imshow(test_img_t.transpose((1, 2, 0)))

In [ ]:
print(nbbox)
test_img = nimg[:, 168:206,  532:571]
test_img_t = inverse_normalize(test_img)/255
plt.imshow(test_img_t.transpose((1, 2, 0)))


In [ ]:
print(label)
print(nlabel)

In [ ]:
print(y_rot)
print(ny_rot)